In [1]:
import json
import os
import pandas as pd

In [2]:
directory = os.getcwd()

In [3]:
with open(directory + "\\response_AAPL.json", "r") as f:
    data = json.load(f)

In [4]:
ts_1 = data

In [5]:
class ScriptData:
    def __init__(self, script):
        self.script = script
        self.fetch_intraday_data(self.script)
        self.df = self.convert_intraday_data(self.data)
        self.df["symbol"] = self.script

    def fetch_intraday_data(self, script):
        # Get json object with the intraday data and another with the call's metadata
        self.data = ts_1
        return self.data
    
    def fetch_historical_data(self):
        # Get json object with the intraday data
        df_csv = pd.read_csv(directory + "\\extended_intraday_GOOGL.csv")
        df_csv = df_csv.rename(columns={'time': 'timestamp'})
        return df_csv
        
    def convert_intraday_data(self, data):
        self.data_list = []
        for timestamp, items in data["Time Series (5min)"].items():
            open_item = float(items["1. open"])
            high_item = float(items["2. high"])
            low_item = float(items["3. low"])
            close_item = float(items["4. close"])
            volume = int(items["5. volume"])
            self.data_list.append([timestamp, open_item, high_item, low_item, close_item, volume])
        df = pd.DataFrame(self.data_list, columns=[ "timestamp", "open", "high", "low", "close", "volume"])
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df

    def indicator1(self, timestamp, dataf, timeperiod= 5):
        indicator_data = []
        final_output = {"timestamp": [], "indicator" : []}
        upcount = 0
        downcount = timeperiod - 1
        while downcount < len(dataf):
            cal_temp = 0
            for i in range(upcount, downcount + 1):
                cal_temp = float(dataf["close"][i]) + cal_temp
            final_temp = cal_temp/timeperiod
            indicator_data.append(final_temp)
            if downcount + 1 >= len(dataf):
                break
            upcount += 1
            downcount += 1
        for i in range(0, len(dataf)):
            final_output["timestamp"].append(dataf["timestamp"][i])
        for i in range(0, len(dataf)):
            if i < timeperiod - 1:
                final_output["indicator"].append("NaN")
            else:
                final_output["indicator"].append(
                    indicator_data[i-timeperiod + 1])
        df_1 = pd.DataFrame(final_output)
        df_1.fillna(value= pd.np.nan, inplace = True)
        return df_1

    def __getitem__(self, index):
        return self.df.loc[index]

    def __setitem__(self, index, value):
        self.df.iloc[index] = value

    def __contains__(self, value):
        return value in self.df['symbol'].values


In [6]:
script_data = ScriptData(ts_1)

In [7]:
script_data.convert_intraday_data(script_data.fetch_intraday_data(ts_1))

timestamp    open     high     low   close  volume
0  2023-02-14 20:00:00  152.79  152.850  152.76  152.84    9429
1  2023-02-14 19:55:00  152.75  152.770  152.73  152.77    3596
2  2023-02-14 19:50:00  152.75  152.760  152.70  152.71    4770
3  2023-02-14 19:45:00  152.84  152.870  152.75  152.76    7379
4  2023-02-14 19:40:00  152.87  152.870  152.84  152.84    1795
..                 ...     ...      ...     ...     ...     ...
95 2023-02-14 12:00:00  151.56  151.830  151.36  151.70  518882
96 2023-02-14 11:55:00  151.53  151.600  151.17  151.55  496456
97 2023-02-14 11:50:00  151.13  151.720  151.10  151.54  494145
98 2023-02-14 11:45:00  151.35  151.386  151.04  151.14  490943
99 2023-02-14 11:40:00  151.21  152.030  151.14  151.33  630530

[100 rows x 6 columns]

In [8]:
script_data.indicator1(timestamp= script_data.df["timestamp"], dataf = script_data.df)

C:\Users\akkin\AppData\Local\Temp\ipykernel_4908\1449781066.py:56: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df_1.fillna(value= pd.np.nan, inplace = True)


timestamp indicator
0  2023-02-14 20:00:00       NaN
1  2023-02-14 19:55:00       NaN
2  2023-02-14 19:50:00       NaN
3  2023-02-14 19:45:00       NaN
4  2023-02-14 19:40:00   152.784
..                 ...       ...
95 2023-02-14 12:00:00    151.84
96 2023-02-14 11:55:00   151.746
97 2023-02-14 11:50:00    151.66
98 2023-02-14 11:45:00   151.522
99 2023-02-14 11:40:00   151.452

[100 rows x 2 columns]

In [9]:
script_data.fetch_historical_data()

timestamp   open     high    low  close  volume
0     2023-02-14 20:00:00  94.18  94.1800  94.11  94.12    9174
1     2023-02-14 19:55:00  94.23  94.2300  94.18  94.18    2882
2     2023-02-14 19:50:00  94.24  94.2400  93.50  93.50   15924
3     2023-02-14 19:45:00  94.26  94.2899  94.20  94.20    5486
4     2023-02-14 19:40:00  94.26  94.3200  94.25  94.32    5043
...                   ...    ...      ...    ...    ...     ...
3853  2023-01-17 04:25:00  91.82  91.8200  91.60  91.72    3784
3854  2023-01-17 04:20:00  91.86  91.8800  91.86  91.88    1980
3855  2023-01-17 04:15:00  91.90  91.9700  91.87  91.87    3610
3856  2023-01-17 04:10:00  91.82  91.9300  91.80  91.90    2016
3857  2023-01-17 04:05:00  92.02  92.1000  91.71  91.79    6551

[3858 rows x 6 columns]

In [10]:
class Strategy:
    
    def __init__(self, script_data):
        self.script_data = script_data

    def fetch_intraday_historical_day_data(self):
        a = self.script_data.fetch_historical_data()
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        return b

    def get_signal(self):
        a = self.script_data.fetch_historical_data()
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        c = {"timestamp": [], "signal" : []}
        for i in range(0, len(b)):
            if float(b["indicator_data"].iloc[i]) < float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("SELL")
            elif float(b["indicator_data"].iloc[i]) > float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("BUY")
            else:
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("NO SIGNAL")
        df_03 = pd.DataFrame(c)
        df_03 = df_03[df_03['signal'] != 'NO SIGNAL'].reset_index(drop=True)
        df_03.to_excel('output.xlsx', index=True)
        return df_03

In [11]:
strategy_data = Strategy(script_data)

In [12]:
strategy_data.fetch_intraday_historical_day_data()

C:\Users\akkin\AppData\Local\Temp\ipykernel_4908\1449781066.py:56: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df_1.fillna(value= pd.np.nan, inplace = True)


timestamp indicator_data  close_data
0     2023-02-14 20:00:00            NaN       94.12
1     2023-02-14 19:55:00            NaN       94.18
2     2023-02-14 19:50:00            NaN       93.50
3     2023-02-14 19:45:00            NaN       94.20
4     2023-02-14 19:40:00         94.064       94.32
...                   ...            ...         ...
3853  2023-01-17 04:25:00           91.7       91.72
3854  2023-01-17 04:20:00          91.73       91.88
3855  2023-01-17 04:15:00         91.772       91.87
3856  2023-01-17 04:10:00          91.82       91.90
3857  2023-01-17 04:05:00         91.832       91.79

[3858 rows x 3 columns]

In [13]:
strategy_data.get_signal()

C:\Users\akkin\AppData\Local\Temp\ipykernel_4908\1449781066.py:56: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df_1.fillna(value= pd.np.nan, inplace = True)


timestamp signal
0     2023-02-14 19:40:00   SELL
1     2023-02-14 19:35:00   SELL
2     2023-02-14 19:30:00   SELL
3     2023-02-14 19:20:00    BUY
4     2023-02-14 19:15:00    BUY
...                   ...    ...
3832  2023-01-17 04:25:00   SELL
3833  2023-01-17 04:20:00   SELL
3834  2023-01-17 04:15:00   SELL
3835  2023-01-17 04:10:00   SELL
3836  2023-01-17 04:05:00    BUY

[3837 rows x 2 columns]

In [15]:
"AAPL" in data["Meta Data"]["2. Symbol"]

True